In [1]:
import pandas as pd
import numpy as np
import pyodbc

In [7]:
srvr = 'SQLMSCP3'
db = 'SLG_Reporting'

cnxn = pyodbc.connect('DRIVER={SQL Server};UID=LGC0069;WSID=LGC-5CG62446DY;Trusted_Connection=Yes;SERVER='+srvr+';DATABASE='+db)
        
query = """SELECT [Name], [UnitCode], [UnitClassification], [Population] FROM Unit"""
        #[Unit.CurrentWaterSewer], [Unit.CurrentElectricFund], [Unit.CurrentGeneralFund], [UnitDetail.AuditYear] FROM Unit, UnitDetail"""
query1 = """SELECT AuditYear, UnitCode, IntergovernmentalDataCode, IntergovernmentalDataValue,
        IntergovernmentalSourceCode FROM UnitDetail"""
query2 = "SELECT [IntergovernmentalDataCode],[IntergovernmentalDataCodeDescription], [AuditYear] FROM IntergovernmentalData"

In [8]:
names = pd.read_sql(query, cnxn)
names.drop_duplicates(inplace = True)


In [9]:
data = pd.read_sql(query1, cnxn)
desc = pd.read_sql(query2, cnxn)
desc.drop_duplicates(inplace = True)
print('done')


done


In [10]:
desc.head()

,IntergovernmentalDataCode,IntergovernmentalDataCodeDescription,AuditYear
0,100,E-EF- Principal paid on LTD,2011
1,100,Electric - Principal Paid - Cash Flow,2012
2,100,Electric - Principal Paid - Cash Flow,2013
3,100,Electric - Principal Paid - Cash Flow,2014
4,100,Electric - Principal Paid - Cash Flow,2015


In [11]:
#data1 = pd.merge(data,desc, how = 'left', on = ['IntergovernmentalDataCode', 'AuditYear'])
data1 = pd.merge(data, names, how = 'left', on = 'UnitCode')

In [12]:
data1 = data1.sort_values('AuditYear', ascending = False).sort_values('UnitCode')

In [13]:
data1.head(10)

,AuditYear,UnitCode,IntergovernmentalDataCode,IntergovernmentalDataValue,IntergovernmentalSourceCode,Name,UnitClassification,Population
0,2007,50001,100,0,CCH,ABERDEEN,A,6726.0
841080,2010,50001,dexp15150,0,AFIR,ABERDEEN,A,6726.0
3733744,2016,50001,d.ws40050,0,AFIR,ABERDEEN,A,6726.0
4564515,2018,50001,385,14294392,CCH,ABERDEEN,A,6726.0
2179943,2013,50001,fexp15130,0,AFIR,ABERDEEN,A,6726.0
130385,2008,50001,107,0,CCH,ABERDEEN,A,6726.0
3736088,2016,50001,d.ws40090,0,AFIR,ABERDEEN,A,6726.0
841684,2010,50001,dexp15160,0,AFIR,ABERDEEN,A,6726.0
2864666,2014,50001,rev10800,49078,AFIR,ABERDEEN,A,6726.0
992421,2011,50001,331,0,CCH,ABERDEEN,A,6726.0


In [14]:
data1 = data1.loc[data1['UnitClassification'].isin(['A','B'])]

In [16]:
len(data1)

4749943

In [43]:
counties = data1.loc[data1['UnitClassification']=='B']

In [44]:
len(counties)

768582

Create a pivot table that shows how the data code changes over the years.

In [6]:
fields = desc.pivot(index = 'IntergovernmentalDataCode', columns = 'AuditYear', values = 'IntergovernmentalDataCodeDescription')

In [7]:
fields.head()

AuditYear,2011,2012,2013,2014,2015,2016,2017,2018
IntergovernmentalDataCode,,,,,,,,
100,E-EF- Principal paid on LTD,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow,Electric - Principal Paid - Cash Flow
101,E-EF- Capital Outlay (SCF),Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow,Electric - Capital Outlay - Cash Flow
102,Tax Collection Rate - Excluding Motor Vehicles,NaN,NaN,Tax collection Rate - Excluding Vehicles,Tax collection Rate - Excluding Vehicles,Tax collection Rate - Excluding Vehicles,Tax collection Rate - Excluding Vehicles,Tax collection Rate - Excluding Vehicles
103,Tax Collection Rate - Motor Vehicles,NaN,NaN,Tax Collection Rate - Vehicles,Tax Collection Rate - Vehicles,Tax Collection Rate - Vehicles,Tax Collection Rate - Vehicles,Tax Collection Rate - Vehicles
104,Hospital-EF- Capital Outlays,NaN,NaN,Hospital-EF- Capital Outlays,NaN,NaN,Hospital-EF- Capital Outlays,Hospital-EF- Capital Outlays


In [8]:
fields.to_csv('./Data/SQL_Data_Fields_Description.csv')

Unstack data so that the unit name and unit code are identifiers over audit years with column names being the data code and values being the data code values. 

In [23]:
datapiv = data1.set_index(['UnitCode', 'Name', 'AuditYear', 'UnitClassification','Population','IntergovernmentalDataCode']).unstack(level=-1)

In [25]:
datapiv.to_csv('./Data/Test_unstack.csv')